### Initialize

In [1]:
# import csv
import json
import os
import random
import requests
import time

# import torch

from tqdm import tqdm
# from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForCausalLM

from constants import DIR_STORED_DATA
from crawler.utils import Logger
from news_with_rationale import NewsWithRationale
from rationale import Rationale
from summarized_news import SummarizedNews
from utils import *
from xai import XAI


logger = Logger(__name__)

# DIR_STORED_DATA = 'stored_data'

if not os.path.exists(DIR_STORED_DATA):
    os.makedirs(DIR_STORED_DATA)



In [2]:

# pipe = pipeline(
#     "text-generation",
#     model="google/gemma-2-2b",
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     device="cuda",
# )

# model_id = "meta-llama/Llama-3.2-1B"

# pipe = pipeline(
#     "text-generation", 
#     model="meta-llama/Llama-3.2-1B",
#     torch_dtype=torch.bfloat16, 
#     # device_map="auto"
#     device="cuda"
# )


## generate rationale

### generate rationale by local model

In [3]:
# load news data

from crawler.crawler_base import News, NewsCrawlerBase

# NEWS_DIR = os.path.join(os.path.dirname(__file__), "crawler/saved_news")
NEWS_DIR = "crawler/saved_news"

def get_news_data() -> list[News]:
    news: list[News] = []
    news_data: list[tuple[str, str]] = [] # (url, file_path)
    with open(os.path.join(NEWS_DIR, "crawled_urls.json"), "r") as f:
        news_data = json.load(f)
        # print(news_data)

    file_list = [file for _, file in news_data]
    for file in file_list:
        news.append(NewsCrawlerBase._parse_file(file))
    return news

# news = get_news_data()
# print(news[0])



In [ ]:
# test rationale generation
import time
news = get_news_data()


doc = news[0].content
# print(f'{len(doc)=}')

doc = doc.replace("\n\n", "\n")
print(f'{len(doc)=}')
# print(doc)

# prompt = get_rationale_prompt_no_gt(doc)
prompt = get_rationale_prompt_chinese(doc, "test")
# print(f'{len(prompt)=}')


s_t = time.time()
output = pipe(prompt, max_length=2048)
e_t = time.time()
print(f'{e_t - s_t=}')
print(output[0]['generated_text'])

### convert row data to SummarizedNews

In [53]:
# convert row data to SummarizedNews


from opencc import OpenCC

cc = OpenCC('s2twp')

def process_str(s: str) -> str:
    return cc.convert(s.replace(" ", ""))  # translate to chinese traditional

summarized_data: list[SummarizedNews] = []

with open('CNewSum_v2/train.simple.label.jsonl', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    count = 0
    # for line in lines:
    for line in tqdm(lines):
        data = json.loads(line)

        article: list[str] = data['article']
        article = [process_str(a) for a in article]
        article_str = '\n'.join(article)

        summary: str = process_str(data['summary'])
        id = data['id']
        # assert len(data['label']) == 1
        # label = data['label'][0]
        label = data['label']
        summarized_data.append(SummarizedNews(article_str, summary, id, label))


print(f'read {len(summarized_data)} summarized news data')

SummarizedNews.save_all(summarized_data)


100%|██████████| 275596/275596 [13:57<00:00, 329.02it/s]

read 275596 summarized news data


### Load data and generate rationale

In [3]:
summarized_news = SummarizedNews.load_all()

x_ai = XAI()
responses_from_x_ai = XAI.get_responses()
rationales_from_x_ai = XAI.load_rationales_from_responses()


[SummarizedNews] [INFO] Loaded 275596 summarized news records
[XAI] [INFO] XAI instance created
[XAI] [INFO] loaded 2056 responses from x_ai responses
[XAI] [ERROR] failed to extract rationale from response: {'code': 'Some resource has been exhausted', 'error': 'h2 protocol error: http2 error'}
[XAI] [ERROR] failed to extract rationale from response: {'code': 'Some resource has been exhausted', 'error': 'h2 protocol error: http2 error'}
[XAI] [ERROR] failed to extract rationale from response: {'id': 'b6d8720b-d319-4324-8b50-2c0afb671e45', 'object': 'chat.completion', 'created': 1735325253, 'model': 'grok-beta', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '很抱歉，但您提供的摘要與文章內容不相符。摘要提到的是達沃斯論壇上的科技展示，而文章講述的是江蘇省鹽城市阜寧縣的自然災害和救災情況。請提供與文章內容相關的摘要，以便我能夠進行分析和摘要撰寫。', 'refusal': None}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 515, 'completion_tokens': 84, 'total_tokens': 599, 'prompt_tokens_details': {'text_tokens': 515, 'audio_tokens': 0, 'image_tokens': 0, 'cached

In [5]:

# corrupted = [472]
for i in range(2000, 2300):
    print(f"processing {i}")
    news = summarized_news[i]
    news_with_rationale = XAI.get_newsWithRationale(news)
    # print(news_with_rationale)
    news_with_rationale.save()



processing 2000
processing 2001
processing 2002
processing 2003
processing 2004
processing 2005
processing 2006
processing 2007
processing 2008
processing 2009
processing 2010
processing 2011
processing 2012
processing 2013
processing 2014
processing 2015
processing 2016
processing 2017
processing 2018
processing 2019
processing 2020
processing 2021
processing 2022
processing 2023
processing 2024
processing 2025
processing 2026
processing 2027
processing 2028
processing 2029
processing 2030
processing 2031
processing 2032
processing 2033
processing 2034
processing 2035
processing 2036
processing 2037
processing 2038
processing 2039
processing 2040
processing 2041
processing 2042
processing 2043
processing 2044
processing 2045
processing 2046
processing 2047
processing 2048
processing 2049
processing 2050
processing 2051
processing 2052
processing 2053
processing 2054
processing 2055
processing 2056
processing 2057


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [14]:
# print(XAI.response)
# print(XAI.rationale)
# print(XAI.responses[-1])
# print(summarized_news[1225].id)
# print(XAI.extract_rationale(XAI.response))
# print(NewsWithRationale(summarized_news[1224], XAI.extract_rationale(XAI.response)))
# print(summarized_news[472])

id: 1512, label: [4, 5]
article: 2015-01-1121:37。
新浪體育。
顯示圖片您的瀏覽器不支援video標籤。
北京時間1月11日,CBA聯賽第31輪天津隊主場迎戰廣東隊。
最終,廣東隊以126-114大勝利天津隊,豪取20連勝。
朱芳雨得到30分5籃板。
本場比賽廣東有6名球員得分上雙。
朱芳雨三分球14中8得到30分。
易建聯26分拜納姆24分。
天津隊5名球員得分上雙。
拜克斯拿下全隊最高的32分,張智涵30分。
賽前廣東隊重要的防守大閘周鵬傷退回到廣東治療。
王仕鵬和朱芳雨同時被提上首發讓廣東隊在一開始就衝擊力十足,第一節他們的三分球就14中9,其中朱芳雨一個人就投進4個三分球。
在第一節的巨大優勢之後廣東隊便打的十分放鬆,縱然該場比賽天津隊偶有發揮,但面對多點開花的廣東隊,天津隊在主場輸掉了比賽。
比賽的第一節好像就是廣東隊的隊內三分練習,半節過去朱芳雨就三分球5投4中,全隊更是投進了9個三分,他們的三分命中率甚至高於天津隊兩分的命中率。
該場比賽廣東隊一共投進了17個三分,命中率高達61%!
其中朱芳雨投進8個,王仕鵬三分球3投3中,拜納姆投進3個,高尚2個,易建聯也進了1個。
今天廣東隊首發中鋒董瀚麟效率也是十分之高,他今天8投6中,罰球4罰3中得到15分9籃板。
面對實力遜於自己的天津隊廣東的命中率十分之高兩分命中率高達53%。
今天天津隊首次派出小將李榮培首發,這名95年的小將在首次首發之後打的十分積極,在首節他就發揮出色得到6分。
他今天得到9分6助攻4籃板3搶斷,極好的彌補了張楠不在的遺憾。
另一名錶現搶眼的是張驥,雖然他的資料不太搶眼,得到11分9籃板。
但是在防守端他卻很好的限制了董瀚麟甚至易建聯,透過身體的接觸使兩名廣東內線十分生氣,出現了許多不該出現的失誤。
國內球員張智涵今天十分積極,屢次下快攻得手,得到30分。
天津隊今天發揮出了自己的水平,無奈對方實力更勝一籌。
下場比賽廣東將主場迎戰同曦隊,而天津隊下場將客場挑戰八一隊。
雙方首發:威廉姆斯。
李榮培。
張驥。
張智涵。
拜克斯廣東首發:朱芳雨。
董瀚麟。
王仕鵬。
易建聯。
拜納姆技術統計
summary: CBA快訊:廣東以126-114大勝利天津,豪取20連勝,朱芳雨得到30分5籃板
Essential Aspects:
['比賽結

### sample test


In [24]:
response = {
    "id": "0f6f9b5a-ba7c-4b01-8054-aa28e855bbac",
    "object": "chat.completion",
    "created": 1735058331,
    "model": "grok-beta",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "核心要素：\n1. 海基會參訪大陸被叫停\n2. 國臺辦的回應\n3. 兩岸交流的影響\n\n三元組：\n- [海基會 | 參訪被叫停 | 大陸]\n- [國臺辦 | 回應 | 海基會參訪被叫停]\n- [臺灣方面 | 原因 | 海基會取消來訪]\n- [兩岸交流 | 受到干擾 | 海基會參訪被叫停]\n\n生成摘要：\n國臺辦回應海基會參訪大陸被叫停，指出是因為臺灣方面的原因，導致海基會取消了這次來訪。我們不希望兩岸正常的交流交往受到干擾。",
                "refusal": None
            },
            "finish_reason": "stop"
        }
    ],
    "usage": {
        "prompt_tokens": 657,
        "completion_tokens": 182,
        "total_tokens": 839,
        "prompt_tokens_details": {
            "text_tokens": 657,
            "audio_tokens": 0,
            "image_tokens": 0,
            "cached_tokens": 0
        }
    },
    "system_fingerprint": "fp_efe33e0791"
}

In [ ]:

# news_with_rationale = NewsWithRationale(summarized_news[201], XAI.extract_rationale(XAI.response))
# # print(news_with_rationale.__str__())
# print(news_with_rationale.__dict__)
# news_with_rationale.save()

# XAI.response = {"id": "e33efe41-e762-4f4d-9174-b1c8ce276b42", "object": "chat.completion", "created": 1735313831, "model": "grok-beta", "choices": [{"index": 0, "message": {"role": "assistant", "content": "核心要素：\n1. 中國和印度的軍工行業發展\n2. 中國軍工行業的效率和先進水平\n3. 印度軍工行業的發展狀況\n\n三元組：\n\n[中國 | 軍工行業發展 | 遠超印度]\n[中國軍工行業 | 效率 | 先進水平的象徵]\n[印度軍工行業 | 發展狀況 | 效率低下]\n[中國 | 軍事研發 | 成功]\n[中國 | 國防預算 | 穩步增加]\n[中國軍工企業 | 參展 | 蘭卡威國際海事及航空航天展]\n\n生成摘要：\n美媒稱，中國軍工發展水平遠超印度。與印度相比，中國軍工行業是效率和先進水平的象徵。中國在軍事研發方面取得了成功，並通過穩步增加國防預算來支持軍工行業的現代化。相比之下，印度軍工行業的發展狀況顯得效率低下。中國軍工企業還積極參加了蘭卡威國際海事及航空航天展，展示其精良裝備。", "refusal": False}, "finish_reason": "stop"}], "usage": {"prompt_tokens": 1259, "completion_tokens": 284, "total_tokens": 1543, "prompt_tokens_details": {"text_tokens": 1259, "audio_tokens": 0, "image_tokens": 0, "cached_tokens": 0}}, "system_fingerprint": "fp_e1b909a5cb"}

# # print(XAI.response)
# rationale = XAI.extract_rationale(XAI.response)
# print(rationale)

### test local rationale generation

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_model_from_pyTorch(model_name: str):
    # load the model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    return tokenizer, model

def generate_text_from_model(tokenizer, model, prompt, max_length=1024):
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()}
    outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

news = get_news_data()
doc = news[0].content

model_name = "google/gemma-2-2b"
tokenizer, model = load_model_from_pyTorch(model_name)
print(f'{model.device=}')
prompt = get_rationale_prompt_no_gt(doc)
print(f'{len(prompt)=}')
output = generate_text_from_model(tokenizer, model, prompt, max_length=3000)
print(f'{len(output)=}')

In [6]:
def f(p: float) -> float:
    return 20000 / (p * 200 + (1 - p) * 40)

print(f(0.485)*40/211.0)

32.24038430538092


In [4]:
prompt = get_rationale_prompt_chinese(summarized_news[2058].article, summarized_news[2058].summary)
print(prompt)
    

給定一份文章及其摘要，完成以下任務：
(1) 根據摘要，提取文章的核心要素。
(2) 對於每個核心要素，檢索詳細的三元組，格式為 [實體1 | 關係 | 實體2]，這些三元組用於構成真實摘要。
(3) 使用檢索到的三元組撰寫一份摘要。核心要素、三元組和撰寫的摘要應該在同一份回應中，並以換行符分隔。所有三元組 [實體1 | 關係 | 實體2] 的長度必須為3（以 "|" 分隔）。
範例：
================範例=================
提示：
[文件]: [文件]
[摘要]: [摘要]
更新：
核心要素：
[核心要素]
三元組：

[實體1_1 | 關係_1 | 實體1_2]
[實體2_1 | 關係_2 | 實體2_2]
[實體3_1 | 關係_3 | 實體3_2]
...
生成摘要：
[摘要]
提示：
[文件]: 大象給小朋友戴禮帽。
距離“六一”國際兒童節還有幾天,錫城園林景區已經準備了豐富的活動讓孩子度過一個歡樂的節日。
其中錫惠景區將放飛五彩繽紛的蝴蝶,無錫動物園海洋館將邀請“美人魚”與海底“藍精靈”共舞,部分景區在兒童節期間還對孩子或陪同家長實行門票減免的優惠措施。
昨日,適逢無錫動物園小象茜茜的9歲生日。
動物園邀請幾個家庭和小象一起度過愉快的生日派對。
如今,小象身高已經2.2米,體重更達1.8噸。
為此動物園為小象製作了一個5層近2米高的水果青草蛋糕。
在高高的生日蛋糕面前,孩子們用西瓜、蘋果、香蕉、南瓜、西紅柿等蔬菜水果將這5層蛋糕精心裝點一番。
孩子們為小象茜茜唱了生日歌,還親手餵它吃了水果青草蛋糕。
作為回禮,小象茜茜不僅表演了拿手的絕活——小象圓舞曲,還用它長長的鼻子為孩子們戴禮帽。
表演完畢後,小象茜茜好好地洗了個澡,還跟在場的孩子們玩了象鼻噴水遊戲。
據瞭解,“六一”節期間,無錫海洋館將邀請美麗的俄羅斯“美人魚”與可愛的海底“藍精靈”共舞,5月30日到6月1日每天表演2場。
5月30日到6。
月7日,錫惠名勝區內中央大廳前的夢幻蝴蝶園將放飛各式各樣的蝴蝶,5月30日到6月1日的上午10點,進入夢幻蝴蝶園的前61名孩子們可以體驗到自己放飛蝴蝶的樂趣。
5月30日到6月7日,錫惠名勝區荷軒長廊內將展出以“多彩童年·愛的憧憬”為主題的蝶戀花少兒繪畫大賽獲獎作品。
園林景區在節日期間還推出很多門票優惠政策。
如動物園5月30